# CP423: Assignment 1

Group Number: 5

Group Members: Abigail Lee (200469770), Kayleigh Habib (200370580) and Myisha Chaudhry (200591740)

In [1]:
# MIGHT NEED TO RUN THIS FOR THE NLTK
#pip install certifi

In [2]:
import certifi
import ssl
import os

os.environ["SSL_CERT_FILE"] = certifi.where()

In [3]:
# inputs
import os
import re

import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kayleighhabib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kayleighhabib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Question 1 : Text Preprocessing
a. Convert all text to lowercase

b. Tokenize the text using NLTK

c. Remove stop words using NLTK

d. Exclude special characters except alphanumeric characters

e. Eliminate singly occurring characters

f. Create a set of all the words

In [4]:
def text_preprocessing_str(query):

    # a. converting to lowercase
    lines = [x.lower() for x in query]

    # join all the lines into one string
    data = "".join(lines)

    # b. using the tokenization package to tokenize the words
    tokenization = word_tokenize(data)

    # remove duplicates - less words to do the next steps with
    newtoken = list(dict.fromkeys(tokenization))

    # c. remove any stop words from the NLTK package
    stop_words_lib = set(stopwords.words("english")) # choosing the english stopwords from the package

    # goes through each word, and checks if it is in the library - if it is not then we keep the word, if it is we don't take it
    stop_words = []
    for word in newtoken:
            if word not in stop_words_lib:
                 stop_words.append(word)

    # print(stop_words)

    # d. Remove special characters
    pattern = "[^a-z0-9]" # pattern NOT with any letter between a-z or any number including 0-9 (could also do A-Z but since we already made everything lower don't need to add that)
    alphanumerictxt = []
    # goes through each index in the list and if it matches the pattern it will put an empty string otherwise will preserve the value
    # strips the empty string afterwards
    for i in stop_words:
        alphanumerictxt.append(re.sub(pattern," ", i).strip())

    # e. eliminating any characters of length 1
    updated_list = []

    # goes through each index in the list and if the length is more than 1 will append to new list
    for i in alphanumerictxt:
        if len(i)>1:
            updated_list.append(i)

    # after all the other steps are completed check again to make sure no duplicated values
    final_list = list(dict.fromkeys(updated_list))


    return final_list

In [5]:
def text_preprocessing_file(file):

    # reading the file line by line
    lines = file.readlines()

    final_list = text_preprocessing_str(lines)

    #f
    return final_list


In [6]:
# import data files
def readfile_fun():
  # takes the names of the files in the folder and creates a list
  file_names = os.listdir("./data")
  # file_names

  # file id (key)
  counter = 1

  #dictionary1 - translates the ID which belong to the file
  dict_words = {}
  # dictionaty2 - file dictionary (id to file name)
  dict_files = {}

  # goes through each file in the list of file names
  for i in file_names:
    # gets the working directory (this may be different for everyone)
    file_path = os.getcwd() + "/data/" + i
    # opens the file in reading mode
    file = open(file_path, "r", encoding="utf-8", errors = "ignore" ) # added the encoding and errors as some files had different encoding detectors (note: with new line can add new param --> newline = '/r/n'))
    # calls the function written for question 1 of the assignment
    words = text_preprocessing_file(file)

    # updates the dictionaries
    dict_words[counter] = words # list of words in this file
    dict_files[counter] = i # list of all the file names

    # increment the counter
    counter+=1

    # closes the file
    file.close()

  # print(dict_files)
  # print(dict_words)

  return dict_words, dict_files

## Question 2 : Inverted Index Implementation
Implement an inverted index data structure for the preprocessed dataset


In [7]:
class HashInvertedIndex:

    # initialize hash table
    def __init__(self, capacity=100): # manually set the capacity
        self.capacity = capacity
        self.table = [[] for _ in range(capacity)]
        self.count = 0

    # finding the corresponding value for the key
    def _index(self, word):
        index = hash(word) % self.capacity
        return index

    # add key-value pair to the hash table
    def insert(self, key, value):

        # compute the hash index for the word
        hash_index = self._index(key)
        for i, kv in enumerate(self.table[hash_index]):
            k, v = kv
            if k == key:
                # add the value to the existing set of values
                v.add(value)
                # add the entry to the table
                self.table[hash_index][i] = (k,v)
                return
        self.table[hash_index].append((key, {value}))

    # retrieve the set of values for a given key, if not found, returns an empty set
    def retrieve(self, key):
        # compute the hash index for a given key
        hash_index = self._index(key)
        for k, v in self.table[hash_index]:
            # if the key if found, return the set
            if k == key:
                return v
        return set()

    # for testing purposes - shows us the string representation of the hash tables
    def __rep__(self):
        return str(self.table)


In [8]:
# build the inverted index - reads the documents and their corresponding words
def create_inverted_index():

    # initialize the hash table/inverted index
    inverted_index = HashInvertedIndex()

    # calls the function above
    # returns list of words in doc and list of docs
    words, files = readfile_fun()

    # fill in the inverted index
    for key, value in words.items():
        for word in value:
            inverted_index.insert(word, key) # inserting counter instead of doc name

    return inverted_index, files, words


In [9]:
# calls the function
inverted_index, file_list, word_list = create_inverted_index()

# prints out the index and the list of file names
# index to first list is hash to the word
for i in range(len(inverted_index.table)):
    #for j in i:
    # prints the hash buckets
    print(inverted_index.table[i])
    # prints the length of each bucket (right now around 400-500 - evenly distributed) - if we want smaller amounts in each bucket increase bucket_size
    #print(len(inverted_index.table[i]))

#print(file_list)


[('bulging', {160, 1, 35, 172, 45, 15, 177, 179, 125, 30}), ('dangerous', {1, 133, 6, 8, 9, 138, 15, 16, 144, 146, 21, 27, 29, 160, 40, 168, 169, 171, 45, 47, 181, 186, 62, 190, 193, 73, 202, 79, 208, 209, 213, 87, 218, 91, 220, 223, 103, 104, 105, 233, 236, 237, 111, 240, 241, 242, 245, 120, 125}), ('presence', {2, 5, 6, 135, 9, 15, 16, 145, 19, 147, 21, 151, 152, 25, 154, 27, 160, 162, 40, 168, 172, 52, 180, 181, 186, 72, 204, 78, 208, 81, 209, 218, 91, 220, 223, 102, 103, 105, 235, 237, 111, 240, 241, 114, 242, 116, 122, 125}), ('seek', {2, 5, 140, 147, 162, 168, 48, 186, 190, 62, 204, 212, 87, 88, 91, 223, 225, 102, 103, 242, 244, 245, 122, 125}), ('beast', {128, 2, 6, 9, 150, 27, 165, 39, 168, 50, 57, 186, 199, 202, 81, 82, 209, 91, 223, 102, 103, 105, 238, 111, 241, 120, 122}), ('guy', {3, 136, 138, 146, 19, 21, 151, 159, 40, 168, 172, 45, 179, 55, 186, 60, 188, 62, 191, 65, 70, 200, 74, 202, 203, 206, 90, 98, 228, 101, 105, 106, 233, 238, 239, 241, 242, 244, 245, 120, 125}), ('d

## Question 3: Query Support
Support the following queries, where x and y would be taken as input from the user

1. x OR y

2. x AND y

3. x AND NOT y

4. x OR NOT y


In [10]:
def query(sentence, operation, InvertedIndex, file):

  #set count of comparisons to 0 at beginning
  #set index to 0
  #set word to the firts word in the sentence
  #set final_list to the inverted index for the first word in the sentence

  comparisons = 0
  index = 0
  word = sentence[index]
  final_list = InvertedIndex.retrieve(word)

  #loop through array of operations
  while index < len(operation):

    #OR
    if operation[index] == "OR":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as UNION will put every value from the min list into the max list removing duplicates
      # using the final_list as the base, counting comparisons made (length of smaller list)
      comparisons += min(len(final_list),len(list1))

      # using the union function to combine the final_list with list1 in order to capture all documents with both words
      final_list = final_list.union(list1)

    #AND
    elif operation[index] == "AND":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as INTERSECTION will keep matching values in the max list removing duplicates
      # using the final_list as the base, counting comparisons made (length of smaller list)
      comparisons += min(len(final_list),len(list1))

      final_list = final_list.intersection(list1)

    elif operation[index] == "AND NOT":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # to get the intersection

      # looking at the minimum of the smallest list, keeping count of the values compared to the base list
      # adding in the values in the intersection of both lists, as values are compared for items that exist in both

      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))

      # removing values that exist in both lists from the list of values for the first word (values that exist for word1 and not word2)
      final_list = final_list - (final_list.intersection(list1))

    elif operation[index] == "OR NOT":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # using the two inverted indexes to find where the first word exists or the second word does not
      # creating temp variable to store list of values where only second word is present
      temp = list1 - (list1.intersection(final_list))

      # looking at the minimum of the smallest list, keeping count of the values compared to the base list
      # adding in the values in the intersection of both lists, as values are compared for items that exist in both
      # adding in the values for comparisons done where only the second word is present

      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))
      comparisons += len(temp)

      # using all document names/values as a universal set, and removing document names/values where only the second word exists
      final_list = set(file.keys()) - temp

    index+=1

  return final_list, comparisons



## Question 4: System Evaluation
Evaluate your system against the set of provided queries.

1. Query #1

- Input Sentence: "lion stood thoughtfully for a moment"

- Input operation sequence: [OR, OR, OR]

- Expected preprocessed query: "lion OR stood OR thoughtfully OR moment"

2. Query #2

- Input Sentence: "telephone, paved, roads"

- Input operation sequence: [OR NOT, AND NOT]

- Expected preprocessed query: "telephone OR NOT paved AND NOT roads


Output should include:
- The number of documents retrieved

- The minimum number of total comparisons made (only for merging the algorithm)

- The list of retrieved document names

In [11]:
text = input("Enter your string: ")
sentence_list = text_preprocessing_str(text)

operations = input("Enter the operations (OR, AND, OR NOT, AND NOT) in CAPS, seperated with commas, NO SPACES between the operations: ")
operations_list = operations.split(",")

print("Sentence: " ,sentence_list)
print("Operations list: ", operations_list)
match_list, comp = query(sentence_list, operations_list, inverted_index,file_list)


print("Number of Matched documents: " , len(match_list))
print("Minimum number of comparisons required: ", comp)
print("List of retreived document names: ")
for match in match_list:
     print(file_list[match])

Sentence:  ['lion', 'stood', 'thoughtfully', 'moment']
Operations list:  ['OR', 'OR', 'OR']
Number of Matched documents:  147
Minimum number of comparisons required:  131
List of retreived document names: 
wlgirl.txt
bulprint.txt
beggars.txt
empty.txt
vainsong.txt
veiledl.txt
angry_ca.txt
foxngrap.txt
darkness.txt
3student.txt
aislesix.txt
pinocch.txt
cooldark.txt
lgoldbrd.txt
healer.txt
jackbstl.txt
history5.txt
bureau.txt
13chil.txt
bulzork1.txt
ab40thv.txt
snow.txt
hotline1.txt
aesopa10.txt
bishop00.txt
bulolli1.txt
blind.txt
bulmrx.txt
goldgoos.txt
7voysinb.txt
timetrav.txt
beautbst.txt
empnclot.txt
enginer.txt
redragon.txt
retrib.txt
buggy.txt
fgoose.txt
hotline3.txt
bulolli2.txt
bluebrd.txt
fantasy.txt
tcoa.txt
rainda.txt
silverb.txt
psf.txt
plescopm.txt
mouslion.txt
emperor3.txt
wisteria.txt
monkking.txt
poplstrm.txt
lure.txt
game.txt
taxnovel.txt
space.txt
abbey.txt
blackp.txt
shoscomb.txt
hellmach.txt
keepmodu.txt
pphamlin.txt
deer.txt
sqzply.txt
aesop11.txt
blh.txt
sick-kid.t